In [9]:
import pandas as pd
import numpy as np
import time 

from sklearn.model_selection import train_test_split
from utils import load_saved_gfp_data, load_saved_mutated_gfp_data, one_hot_decode, one_hot_encode, normalize

In [11]:
print("Loading data...")
start_time = time.time()
X_train, X_test, y_train, y_test = load_saved_gfp_data()
mutated_df = load_saved_mutated_gfp_data()
print("Finished loading data in {0:.2f} seconds".format(time.time() - start_time))

Loading data...
Finished loading data in 3.85 seconds
